# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2020 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2020 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [17]:
# YOUR CHANGES HERE



import pandas as pd

IN_PATH  = "strawberry-prices.tsv"


df = pd.read_csv(IN_PATH, sep="\t")
date_col = "month" if "month" in df.columns else df.columns[0]
df[date_col] = pd.to_datetime(df[date_col])
df = df.rename(columns={c:"price" for c in df.columns if c != date_col})
df = df[[date_col, "price"]].dropna()
df["year"] = df[date_col].dt.year
df["m"]    = df[date_col].dt.month

seasonal_mean = (
    df.query("2020 <= year <= 2023")
      .groupby("m")["price"]
      .mean()
)

months_2024 = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
pred = pd.DataFrame({
    "month": months_2024,
    "m": months_2024.month
})
pred["price"] = pred["m"].map(seasonal_mean)

pred_out = pred[["month","price"]].copy()
pred_out["month"] = pred_out["month"].dt.strftime("%Y-%m")
pred_out.to_csv("strawberry-backtest.tsv", sep="\t", index=False)

print(f"Saved: {"strawberry-backtest.tsv"}")



Saved: strawberry-backtest.tsv


Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [16]:
# YOUR CHANGES HERE

...


import pandas as pd

PRICES_FILE   = "strawberry-prices.tsv"



actual = pd.read_csv(PRICES_FILE, sep="\t")
actual.columns = [c.strip().lower() for c in actual.columns]
date_col = "month" if "month" in actual.columns else actual.columns[0]
actual[date_col] = pd.to_datetime(actual[date_col])
actual = actual.rename(columns={c:"price" for c in actual.columns if c != date_col})
actual_2024 = actual[actual[date_col].dt.year == 2024].copy()

pred = pd.read_csv("strawberry-backtest.tsv", sep="\t")
pred["month"] = pd.to_datetime(pred["month"])

merged = pd.merge(actual_2024, pred, left_on=date_col, right_on="month", how="inner", suffixes=("_actual", "_pred"))
merged["residual"] = merged["price_actual"] - merged["price_pred"]

mean_resid = merged["residual"].mean()
std_resid  = merged["residual"].std()

pd.DataFrame({"mean":[mean_resid], "std":[std_resid]}).to_csv("backtest-accuracy.tsv", sep="\t", index=False)
print(f"Saved residual mean/std to: {"backtest-accuracy.tsv"}")
print(merged[["month", "price_actual", "price_pred", "residual"]])
print(f"\nMean residual: {mean_resid:.4f}")
print(f"Std residual:  {std_resid:.4f}")

Saved residual mean/std to: backtest-accuracy.tsv
        month  price_actual  price_pred  residual
0  2024-01-01         5.055    4.362750  0.692250
1  2024-02-01         4.264    4.091250  0.172750
2  2024-03-01         3.742    3.688750  0.053250
3  2024-04-01         3.576    3.787200 -0.211200
4  2024-05-01         3.237    3.526250 -0.289250
5  2024-06-01         2.977    3.275000 -0.298000
6  2024-07-01         3.116    3.194000 -0.078000
7  2024-08-01         3.347    3.489750 -0.142750
8  2024-09-01         3.742    3.581750  0.160250
9  2024-10-01         3.718    3.926000 -0.208000
10 2024-11-01         4.420    4.398000  0.022000
11 2024-12-01         4.857    4.788575  0.068425

Mean residual: -0.0049
Std residual:  0.2746


Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2020 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [15]:
# YOUR CHANGES HERE


import pandas as pd

IN_PATH  = "strawberry-prices.tsv"


df = pd.read_csv(IN_PATH, sep="\t")
date_col = "month" if "month" in df.columns else df.columns[0]
df[date_col] = pd.to_datetime(df[date_col])
df = df.rename(columns={c: "price" for c in df.columns if c != date_col})
df = df[[date_col, "price"]].dropna()
df["year"] = df[date_col].dt.year
df["m"]    = df[date_col].dt.month

seasonal_mean = (
    df.query("2020 <= year <= 2024")
      .groupby("m")["price"]
      .mean()
)

months_2025 = pd.date_range("2025-01-01", "2025-12-01", freq="MS")
forecast = pd.DataFrame({
    "month": months_2025,
    "m": months_2025.month
})
forecast["price"] = forecast["m"].map(seasonal_mean)

forecast_out = forecast[["month", "price"]].copy()
forecast_out["month"] = forecast_out["month"].dt.strftime("%Y-%m")
forecast_out.to_csv("strawberry-forecast.tsv", sep="\t", index=False)

print(f"Saved: {"strawberry-forecast.tsv"}")

...

Saved: strawberry-forecast.tsv


Ellipsis

Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

Submit "timings.tsv" in Gradescope.

In [14]:
# analyze_timings.py
# Computes profits for all 2025 buy/sell month pairs and saves timings.tsv

import pandas as pd
from pathlib import Path

PRICES_FILE   = Path("strawberry-prices.tsv")
FORECAST_FILE = Path("strawberry-forecast.tsv")
OUT_FILE      = Path("timings.tsv")

FREEZE_COST = 0.20          
STORAGE_PER_MONTH = 0.10    
DISCOUNT = 0.10             
BUDGET = 1_000_000          
# -------------------------------------------------------

def load_or_build_forecast_2025():
    """Load strawberry-forecast.tsv or build it using the Part-1 seasonal-mean method (2020–2024)."""
    if FORECAST_FILE.exists():
        fc = pd.read_csv(FORECAST_FILE, sep="\t")
        fc["month"] = pd.to_datetime(fc["month"])
        return fc[["month", "price"]]

    # If forecast file doesn't exist, rebuild it
    df = pd.read_csv(PRICES_FILE, sep="\t")
    date_col = "month" if "month" in df.columns else df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.rename(columns={c: "price" for c in df.columns if c != date_col})
    df = df[[date_col, "price"]].dropna()
    df["year"] = df[date_col].dt.year
    df["m"] = df[date_col].dt.month

    seasonal_mean = (
        df.query("2020 <= year <= 2024")
          .groupby("m")["price"]
          .mean()
    )

    months_2025 = pd.date_range("2025-01-01", "2025-12-01", freq="MS")
    fc = pd.DataFrame({"month": months_2025, "m": months_2025.month})
    fc["price"] = fc["m"].map(seasonal_mean)
    return fc[["month", "price"]]

def main():
    forecast = load_or_build_forecast_2025().reset_index(drop=True)

    results = []
    for i in range(len(forecast) - 1):
        for j in range(i + 1, len(forecast)):
            buy_month  = forecast.loc[i, "month"]
            sell_month = forecast.loc[j, "month"]
            buy_price  = float(forecast.loc[i, "price"])
            sell_price = float(forecast.loc[j, "price"])

            m_gap = (sell_month.to_period("M") - buy_month.to_period("M")).n

            revenue_per_pint = (1 - DISCOUNT) * sell_price
            total_cost_per_pint = buy_price + FREEZE_COST + STORAGE_PER_MONTH * m_gap
            profit_per_pint = revenue_per_pint - total_cost_per_pint

            pints = int(BUDGET // total_cost_per_pint) if total_cost_per_pint > 0 else 0
            expected_profit = profit_per_pint * pints

            results.append({
                "buy_month":  buy_month.strftime("%Y-%m"),
                "sell_month": sell_month.strftime("%Y-%m"),
                "pints_purchased": pints,
                "expected_profit": expected_profit
            })

    df_out = pd.DataFrame(results)

    out_path = OUT_FILE
    df_out.to_csv(out_path, sep="\t", index=False)
    print(f"Saved results to: {out_path.resolve()}\n")

    print("Top 5 most profitable scenarios:")
    print(df_out.sort_values("expected_profit", ascending=False).head(5))

if __name__ == "__main__":
    main()


Saved results to: /workspaces/dx704-project-02/timings.tsv

Top 5 most profitable scenarios:
   buy_month sell_month  pints_purchased  expected_profit
55   2025-07    2025-12           257838    114385.703292
50   2025-06    2025-12           249041     76364.437994
59   2025-08    2025-12           246232     64225.677488
62   2025-09    2025-12           243084     50618.353656
54   2025-07    2025-11           264662     48634.289120


## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [ ]:
# YOUR CHANGES HERE




import pandas as pd
from pathlib import Path


TIMINGS_FILE   = Path("timings.tsv")
FORECAST_FILE  = Path("strawberry-forecast.tsv")   
PRICES_FILE    = Path("strawberry-prices.tsv")     
BACKTEST_FILE  = Path("strawberry-backtest.tsv")   
OUT_FILE       = Path("check.tsv")

FREEZE_COST = 0.20           
STORAGE_PER_MONTH = 0.10     
DISCOUNT = 0.10              

timings = pd.read_csv(TIMINGS_FILE, sep="\t")
best_idx = timings["expected_profit"].idxmax()
best = timings.loc[best_idx].copy()

best_profit = float(best["expected_profit"])
buy_month  = str(best["buy_month"])
sell_month = str(best["sell_month"])
pints      = int(best["pints_purchased"])

actual = pd.read_csv(PRICES_FILE, sep="\t")
actual.columns = [c.strip().lower() for c in actual.columns]
date_col = "month" if "month" in actual.columns else actual.columns[0]
actual[date_col] = pd.to_datetime(actual[date_col])
actual = actual.rename(columns={c: "price" for c in actual.columns if c != date_col})
actual_2024 = actual[actual[date_col].dt.year == 2024][[date_col, "price"]].copy()

pred = pd.read_csv(BACKTEST_FILE, sep="\t")
pred["month"] = pd.to_datetime(pred["month"])

m = actual_2024.merge(pred, left_on=date_col, right_on="month", how="inner", suffixes=("_actual", "_pred"))
m["residual"] = m["price_actual"] - m["price_pred"]
std_resid = float(m["residual"].std())

fc = pd.read_csv(FORECAST_FILE, sep="\t")
fc["month"] = pd.to_datetime(fc["month"])

buy_price  = float(fc.loc[fc["month"].dt.strftime("%Y-%m") == buy_month, "price"].values[0])
sell_price = float(fc.loc[fc["month"].dt.strftime("%Y-%m") == sell_month, "price"].values[0])

gap = (pd.Period(sell_month, freq="M") - pd.Period(buy_month, freq="M")).n

total_cost_per_pint = buy_price + FREEZE_COST + STORAGE_PER_MONTH * gap
sell_price_adj = max(sell_price - std_resid, 0.0)  # don't allow negative
revenue_per_pint_adj = (1 - DISCOUNT) * sell_price_adj
profit_per_pint_adj = revenue_per_pint_adj - total_cost_per_pint

one_std_profit = float(profit_per_pint_adj * pints)

pd.DataFrame([{"best_profit": best_profit, "one_std_profit": one_std_profit}]).to_csv(
    OUT_FILE, sep="\t", index=False
)

print(f"Saved: {OUT_FILE.resolve()}")
print(f"Best profit: {best_profit:,.2f}")
print(f"Profit with sell price -1σ: {one_std_profit:,.2f}")


Saved: /workspaces/dx704-project-02/check.tsv
Best profit: 114,385.70
Profit with sell price -1σ: 50,671.49


Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [ ]:
from pathlib import Path


ACK_FILE = Path("acknowledgments.txt")
ACK_TEXT = "My dad helped me with this"


def main():
    ACK_FILE.write_text(ACK_TEXT)
    print(f"Saved acknowledgments to: {ACK_FILE.resolve()}")

if __name__ == "__main__":
    main()

Saved acknowledgments to: /workspaces/dx704-project-02/acknowledgments.txt


## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.